In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0", device_map="auto")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rot

In [ ]:
messages = [
    {"role": "user", "content": "Who was the winner of 2022 world cup?"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

The winner of the 2022 FIFA World Cup is Argentina, who defeated France in the final match on July 18, 2022, in Lusail, Q


In [ ]:
import torch

# total trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params:,}")

# total parameters (trainable + frozen)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:,}")


Trainable parameters: 1,100,048,384
Total parameters: 1,100,048,384


In [ ]:
messages = [
    {"role": "user", "content": "Who was the winner of the 2024 Olympic men’s football tournament?"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

The winner of the 2024 Olympics men's football is yet to be determined. The 2024 Olympics will be held in Paris, France, and the host nation has


In [ ]:
# Reference https://www.fifa.com/en/tournaments/olympicgames/paris2024/articles/medal-winners-mens-tournament

text="""
The 2024 edition of the Olympic Men’s Football Tournament has now concluded, with Spain taking gold at one of the world’s greatest sporting events.

Football was first included at the Olympic Games at Paris 1900 – and Ferenc Puskas, Lionel Messi and Neymar are among a wealth of iconic figures who have lit up the competition over the years.

After France hosted the tournament again in 2024, FIFA lists the most successful nations in the event's rich history.
"""

In [ ]:
from peft import LoraConfig, get_peft_model, PeftModel


# -------------------------------------------------------
# LoRA Config
# -------------------------------------------------------
lora_config = LoraConfig(
    r=8,  #controls the size and capacity of the LoRA adapter
    lora_alpha=16,   #rescales the LoRA update before adding it to the base weight
    lora_dropout=0.1,  # Dropout applied only on the LoRA update, not the base model
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # correct for most LLMs
)

model = get_peft_model(model, lora_config)



/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [ ]:
# total parameters (trainable + frozen)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:,}")

# total trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params:,}")


Trainable parameters: 2,252,800
Total parameters: 1,102,301,184


In [ ]:
lora_params = sum(
    p.numel() for n, p in model.named_parameters() if "lora" in n
)
print(f"LoRA parameters: {lora_params:,}")


LoRA parameters: 2,252,800


In [ ]:
model

PeftModel(
  (base_model): LoraModel(
    (model): PeftModel(
      (base_model): LoraModel(
        (model): LlamaForCausalLM(
          (model): LlamaModel(
            (embed_tokens): Embedding(32000, 2048)
            (layers): ModuleList(
              (0-21): 22 x LlamaDecoderLayer(
                (self_attn): LlamaAttention(
                  (q_proj): lora.Linear(
                    (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=2048, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=2048, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    (lora_embedding_B): Param

In [ ]:
from datasets import Dataset
from transformers import Trainer, TrainingArguments

# -------------------------------------------------------
# Dataset — plain text → one sequence
# -------------------------------------------------------

dataset = Dataset.from_dict({"text": [text]})

# -------------------------------------------------------
# Tokenization
# -------------------------------------------------------
def tokenize(batch):
    out = tokenizer(
        batch["text"],
        truncation=True,
        max_length=512,
    )
    out["labels"] = out["input_ids"].copy()
    return out

train_dataset = dataset.map(tokenize, batched=True, remove_columns=["text"])

# -------------------------------------------------------
# TrainingArguments
# -------------------------------------------------------
args = TrainingArguments(
    report_to="none",
    output_dir="outputs",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,   # matters when dataset is tiny
    num_train_epochs=30,             # tiny dataset → need MORE epochs
    learning_rate=5e-4,              # high LR for tiny dataset
    lr_scheduler_type="constant",
    logging_steps=1,
    save_steps=10,
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
)

trainer.train()


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


Step,Training Loss
1,2.271600
2,2.088400
3,1.848900
4,1.578100
5,1.311700
6,1.052900
7,0.799100
8,0.650600
9,0.386600
10,0.231000


In [ ]:
# -------------------------------------------------------
# Save LoRA adapter
# -------------------------------------------------------
model.save_pretrained("outputs/lora")


# Load LoRA adapter
The following is the code for loading the trained LoRA adapter with the TinyLlama model, which could be run in another runtime. The training code above have already been using the adapter with the model so no need to load LoRA again.  

In [ ]:
from peft import PeftModel

adapter_path = "lora_output"

# Load LoRA adapter on top of the base model
trained_model = PeftModel.from_pretrained(model, adapter_path)

trained_model.eval()

In [ ]:
content = "Who was the winner of the 2024 Olympic men’s football tournament?"
messages = [
    {"role": "user", "content": content},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(trained_model.device)

outputs = trained_model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

In [ ]:
# Merge model and adapter

merged_model = model.merge_and_unload()
merged_model.save_pretrained("merged_model/")
tokenizer.save_pretrained("merged_model/")


In [ ]:
# Login HuggingFace
from huggingface_hub import login
login()


In [ ]:
from huggingface_hub import create_repo

create_repo("model_name", private=False)  # 可以改成 public


In [ ]:
from huggingface_hub import upload_folder

upload_folder(
    folder_path="merged_model/",
    repo_id="account_name/model_name",
)
